In [1]:
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

In [3]:
"""layers.py"""
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        # init
        super(GraphConvolution, self).__init__()
        # input parameters
        self.in_features = in_features
        self.out_features = out_features
        # model parameters
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        # model parameters initialization
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        # forward: output = adj * input * weight + bias
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        # 
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [5]:
# build class for GraphConvolution layer
gcn_layer = GraphConvolution(10, 10)
print(gcn_layer)

GraphConvolution (10 -> 10)


In [7]:
import torch.nn as nn
import torch.nn.functional as F

In [10]:
"""models.py"""
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        # init
        super(GCN, self).__init__()
        # two gc layers
        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        # forward: model
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [11]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

from pygcn.utils import load_data, accuracy
from pygcn.models import GCN

In [13]:
"""training"""

'training'